# Commercial Card Portfolio Analysis

**Objective**: Analyze commercial card portfolio performance, customer behavior, and identify growth opportunities

**Key Areas**:
- Portfolio performance metrics
- Customer segmentation analysis  
- Transaction pattern analysis
- Geographic and category insights

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 1. Data Loading and Initial Exploration

In [ ]:
transactions_df = pd.read_csv('../data/processed/transactions_processed.csv')
customers_df = pd.read_csv('../data/processed/customers_enhanced.csv')
campaigns_df = pd.read_csv('../data/processed/campaigns_summary.csv')

print(f"Transactions: {len(transactions_df):,} records")
print(f"Customers: {len(customers_df):,} records")
print(f"Campaigns: {len(campaigns_df):,} records")

transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'])
customers_df['customer_since_date'] = pd.to_datetime(customers_df['customer_since_date'])

## 2. Portfolio Performance Overview

In [ ]:
monthly_performance = transactions_df.groupby(transactions_df['transaction_date'].dt.to_period('M')).agg({
    'transaction_amount': ['sum', 'mean', 'count'],
    'customer_id': 'nunique'
}).round(2)

monthly_performance.columns = ['Total_Spend', 'Avg_Transaction', 'Transaction_Count', 'Active_Customers']
monthly_performance = monthly_performance.reset_index()
monthly_performance['transaction_date'] = monthly_performance['transaction_date'].astype(str)

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Monthly Total Spend', 'Monthly Active Customers', 
                   'Average Transaction Amount', 'Transaction Count'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

fig.add_trace(go.Scatter(x=monthly_performance['transaction_date'], y=monthly_performance['Total_Spend'],
                        name='Total Spend', line=dict(color='blue')), row=1, col=1)

fig.add_trace(go.Scatter(x=monthly_performance['transaction_date'], y=monthly_performance['Active_Customers'],
                        name='Active Customers', line=dict(color='green')), row=1, col=2)

fig.add_trace(go.Scatter(x=monthly_performance['transaction_date'], y=monthly_performance['Avg_Transaction'],
                        name='Avg Transaction', line=dict(color='red')), row=2, col=1)

fig.add_trace(go.Scatter(x=monthly_performance['transaction_date'], y=monthly_performance['Transaction_Count'],
                        name='Transaction Count', line=dict(color='purple')), row=2, col=2)

fig.update_layout(height=600, title_text="Portfolio Performance Trends", showlegend=False)
fig.show()

## 3. Customer Segmentation Analysis

In [ ]:
segment_analysis = customers_df.groupby('customer_segment').agg({
    'customer_id': 'count',
    'total_spend': ['sum', 'mean'],
    'avg_monthly_spend': 'mean',
    'transaction_count': 'mean',
    'credit_utilization': 'mean',
    'customer_tenure_months': 'mean'
}).round(2)

segment_analysis.columns = ['Customer_Count', 'Total_Spend', 'Avg_Customer_Spend', 
                           'Avg_Monthly_Spend', 'Avg_Transactions', 'Avg_Utilization', 'Avg_Tenure']

segment_analysis['Spend_Percentage'] = (segment_analysis['Total_Spend'] / segment_analysis['Total_Spend'].sum() * 100).round(1)
segment_analysis['Customer_Percentage'] = (segment_analysis['Customer_Count'] / segment_analysis['Customer_Count'].sum() * 100).round(1)

print("Customer Segment Performance:")
display(segment_analysis)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

segment_analysis['Customer_Percentage'].plot(kind='pie', ax=ax1, autopct='%1.1f%%')
ax1.set_title('Customer Distribution by Segment')
ax1.set_ylabel('')

segment_analysis['Spend_Percentage'].plot(kind='pie', ax=ax2, autopct='%1.1f%%')
ax2.set_title('Spend Distribution by Segment')
ax2.set_ylabel('')

segment_analysis['Avg_Customer_Spend'].plot(kind='bar', ax=ax3)
ax3.set_title('Average Customer Spend by Segment')
ax3.set_ylabel('Spend ($)')
ax3.tick_params(axis='x', rotation=45)

segment_analysis['Avg_Utilization'].plot(kind='bar', ax=ax4, color='orange')
ax4.set_title('Average Credit Utilization by Segment')
ax4.set_ylabel('Utilization (%)')
ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Transaction Pattern Analysis

In [ ]:
category_analysis = transactions_df.groupby('merchant_category').agg({
    'transaction_amount': ['sum', 'mean', 'count'],
    'customer_id': 'nunique'
}).round(2)

category_analysis.columns = ['Total_Spend', 'Avg_Transaction', 'Transaction_Count', 'Unique_Customers']
category_analysis = category_analysis.sort_values('Total_Spend', ascending=False)
category_analysis['Spend_Percentage'] = (category_analysis['Total_Spend'] / category_analysis['Total_Spend'].sum() * 100).round(1)

print("Top 10 Merchant Categories by Spend:")
display(category_analysis.head(10))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

top_categories = category_analysis.head(10)
top_categories['Total_Spend'].plot(kind='barh', ax=ax1)
ax1.set_title('Top 10 Categories by Total Spend')
ax1.set_xlabel('Total Spend ($)')

top_categories['Avg_Transaction'].plot(kind='barh', ax=ax2, color='green')
ax2.set_title('Top 10 Categories by Average Transaction')
ax2.set_xlabel('Average Transaction ($)')

plt.tight_layout()
plt.show()

## 5. Geographic Analysis

In [ ]:
geo_analysis = customers_df.groupby('geographic_region').agg({
    'customer_id': 'count',
    'total_spend': ['sum', 'mean'],
    'avg_monthly_spend': 'mean',
    'credit_utilization': 'mean'
}).round(2)

geo_analysis.columns = ['Customer_Count', 'Total_Spend', 'Avg_Customer_Spend', 'Avg_Monthly_Spend', 'Avg_Utilization']
geo_analysis = geo_analysis.sort_values('Total_Spend', ascending=False)

print("Geographic Performance Analysis:")
display(geo_analysis)

fig = px.bar(geo_analysis.reset_index(), x='geographic_region', y='Total_Spend',
             title='Total Spend by Geographic Region',
             labels={'Total_Spend': 'Total Spend ($)', 'geographic_region': 'Region'})
fig.show()

fig2 = px.scatter(geo_analysis.reset_index(), x='Customer_Count', y='Avg_Customer_Spend',
                 size='Total_Spend', hover_name='geographic_region',
                 title='Customer Count vs Average Spend by Region',
                 labels={'Customer_Count': 'Number of Customers', 'Avg_Customer_Spend': 'Average Customer Spend ($)'})
fig2.show()

## 6. Customer Lifetime Value Analysis

In [ ]:
customers_df['clv_quartile'] = pd.qcut(customers_df['total_spend'], 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

clv_analysis = customers_df.groupby('clv_quartile').agg({
    'customer_id': 'count',
    'total_spend': ['sum', 'mean'],
    'customer_tenure_months': 'mean',
    'transaction_count': 'mean',
    'avg_monthly_spend': 'mean'
}).round(2)

clv_analysis.columns = ['Customer_Count', 'Total_Spend', 'Avg_CLV', 'Avg_Tenure', 'Avg_Transactions', 'Avg_Monthly_Spend']
clv_analysis['CLV_Contribution'] = (clv_analysis['Total_Spend'] / clv_analysis['Total_Spend'].sum() * 100).round(1)

print("Customer Lifetime Value Analysis:")
display(clv_analysis)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

clv_analysis['CLV_Contribution'].plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('CLV Contribution by Quartile (%)')
ax1.set_ylabel('Contribution (%)')
ax1.tick_params(axis='x', rotation=0)

clv_analysis['Avg_CLV'].plot(kind='bar', ax=ax2, color='lightcoral')
ax2.set_title('Average CLV by Quartile')
ax2.set_ylabel('Average CLV ($)')
ax2.tick_params(axis='x', rotation=0)

clv_analysis['Avg_Tenure'].plot(kind='bar', ax=ax3, color='lightgreen')
ax3.set_title('Average Tenure by CLV Quartile')
ax3.set_ylabel('Tenure (Months)')
ax3.tick_params(axis='x', rotation=0)

clv_analysis['Avg_Monthly_Spend'].plot(kind='bar', ax=ax4, color='gold')
ax4.set_title('Average Monthly Spend by CLV Quartile')
ax4.set_ylabel('Monthly Spend ($)')
ax4.tick_params(axis='x', rotation=0)

plt.tight_layout()
plt.show()

## 7. Key Insights and Recommendations

In [ ]:
insights = {
    'Portfolio Size': f"{len(customers_df):,} active customers",
    'Total Portfolio Spend': f"${customers_df['total_spend'].sum()/1e6:.1f}M",
    'Average Customer Value': f"${customers_df['total_spend'].mean():.0f}",
    'Top Segment by Spend': segment_analysis.index[0],
    'Top Geographic Region': geo_analysis.index[0],
    'Top Category by Volume': category_analysis.index[0],
    'High Value Customers (Q4)': f"{clv_analysis.loc['Q4', 'CLV_Contribution']:.1f}% of total spend",
    'Average Credit Utilization': f"{customers_df['credit_utilization'].mean():.1f}%"
}

print("=" * 50)
print("KEY PORTFOLIO INSIGHTS")
print("=" * 50)
for key, value in insights.items():
    print(f"{key}: {value}")

print("\n" + "=" * 50)
print("STRATEGIC RECOMMENDATIONS")
print("=" * 50)
print("1. Focus retention efforts on Q4 customers (high-value segment)")
print("2. Expand marketing in top-performing geographic regions")
print("3. Develop category-specific rewards for top spending categories")
print("4. Implement credit limit optimization for underutilized customers")
print("5. Create targeted cross-sell campaigns for single-product customers")
print("6. Develop early warning system for churn prevention")
print("7. Optimize pricing strategy based on customer segment performance")